In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress
from yelp.client import Client
import json
from ratelimit import limits
import requests
import csv
import random
from keys import my_api_key


df = pd.read_csv("../sub-est2019_all.csv", engine='python')
rural_city = df[(df["POPESTIMATE2019"]< 50000) & (df["POPESTIMATE2019"] > 1000)]
rural_df = rural_city[["NAME", "STNAME", "POPESTIMATE2019"]]
urban_city = df[df["POPESTIMATE2019"]>= 50000]
urban_df = urban_city[["NAME", "STNAME", "POPESTIMATE2019"]]
urban_index = urban_df.loc[urban_df["NAME"] == urban_df["STNAME"]].index
urban_clean = urban_df.drop(urban_index)
random_urban_city = urban_clean[["NAME", "STNAME","POPESTIMATE2019"]].sample(n=100, random_state=1)
random_rural_city = rural_df[["NAME", "STNAME", "POPESTIMATE2019"]].sample(n=100, random_state=1)


In [7]:
headers = {'Authorization': 'Bearer %s' % my_api_key}

rural_dict = {
    "Searched": [],
    "City": [],
    "State": [],
    "Lat": [],
    "Lng": [],
    "Restaurant": [],
    "Category": [],
    "Transactions":[]
}

url='https://api.yelp.com/v3/businesses/search'
 
for city in random_rural_city["NAME"]:
    params = {
        'location': city,
        'radius': 4000
             }
    response = requests.get(url, params = params, headers = headers).json()
    for business in range(len(response["businesses"])):
        try:
            rural_dict["Searched"].append(city)
            rural_dict["City"].append(response["businesses"][business]["location"]["city"])
            rural_dict["State"].append(response["businesses"][business]["location"]["state"])
            rural_dict["Lat"].append(response["businesses"][business]["coordinates"]["latitude"])
            rural_dict["Lng"].append(response["businesses"][business]["coordinates"]["longitude"])
            rural_dict["Restaurant"].append(response["businesses"][business]["name"])
            rural_dict["Category"].append([response["businesses"][business]["categories"][i]["title"] for i in range(len(response["businesses"][business]["categories"]))])
            rural_dict["Transactions"].append(response["businesses"][business]["transactions"])
        except KeyError:
            rural_dict["Searched"].append("No City Found")
            rural_dict["City"].append("No City Found")
            rural_dict["State"].append("No City Found")
            rural_dict["Lat"].append("No City Found")
            rural_dict["Lng"].append("No City Found")
            rural_dict["Restaurant"].append("No City Found")
            rural_dict["Category"].append("No City Found")
            rural_dict["Transactions"].append("No City Found")

In [8]:
rural = pd.DataFrame.from_dict(rural_dict, orient='index')
rural = rural.transpose()
rural.head()

,Searched,City,State,Lat,Lng,Restaurant,Category,Transactions
0,Montezuma township,Milton,IL,39.5637,-90.6448,Dink's Diner,"[Desserts, Salad, Sandwiches]",[]
1,Montezuma township,Pearl,IL,39.5638,-90.6491,Mangley's,[American (Traditional)],[]
2,Genoa City village,Genoa City,WI,42.4969,-88.3308,Fitzgerald's Genoa Junction,"[Seafood, Barbeque]",[]
3,Genoa City village,Genoa City,WI,42.4986,-88.3138,Herner's Hideaway,"[Bars, American (New)]",[]
4,Genoa City village,Genoa City,WI,42.5012,-88.3194,Wayne's Pizza,"[Pizza, Hot Dogs, American (Traditional)]",[]


In [11]:
headers = {'Authorization': 'Bearer %s' % my_api_key}

urban_dict = {
    "Searched": [],
    "City": [],
    "State": [],
    "Lat": [],
    "Lng": [],
    "Restaurant": [],
    "Category": [],
    "Transactions":[]
}

url='https://api.yelp.com/v3/businesses/search'
 
for city in random_urban_city["NAME"]:
    params = {
        'location': city,
        'radius': 4000
             }
    response = requests.get(url, params = params, headers = headers).json()
    for business in range(len(response["businesses"])):
        try:
            urban_dict["Searched"].append(city)
            urban_dict["City"].append(response["businesses"][business]["location"]["city"])
            urban_dict["State"].append(response["businesses"][business]["location"]["state"])
            urban_dict["Lat"].append(response["businesses"][business]["coordinates"]["latitude"])
            urban_dict["Lng"].append(response["businesses"][business]["coordinates"]["longitude"])
            urban_dict["Restaurant"].append(response["businesses"][business]["name"])
            urban_dict["Category"].append([response["businesses"][business]["categories"][i]["title"] for i in range(len(response["businesses"][business]["categories"]))])
            urban_dict["Transactions"].append(response["businesses"][business]["transactions"])
        except KeyError:
            urban_dict["Searched"].append("No City Found")
            urban_dict["City"].append("No City Found")
            urban_dict["State"].append("No City Found")
            urban_dict["Lat"].append("No City Found")
            urban_dict["Lng"].append("No City Found")
            urban_dict["Restaurant"].append("No City Found")
            urban_dict["Category"].append("No City Found")
            urban_dict["Transactions"].append("No City Found")

In [18]:
urban = pd.DataFrame.from_dict(urban_dict, orient='index')
urban = urban.transpose()
urban.head()

,Searched,City,State,Lat,Lng,Restaurant,Category,Transactions
0,Brentwood city,Brentwood,CA,37.9328,-121.695,Dino's Sandwich Shop,"[Delis, Sandwiches, Salad]",[delivery]
1,Brentwood city,Brentwood,CA,37.9317,-121.695,Zephyr Grill & Bar,"[American (Traditional), Seafood, Steakhouses]","[delivery, pickup]"
2,Brentwood city,Brentwood,CA,37.9372,-121.699,Big House Beans,"[Coffee & Tea, Coffee Roasteries]",[delivery]
3,Brentwood city,Brentwood,CA,37.9369,-121.698,TeaZenTea,"[Coffee & Tea, Bubble Tea, Juice Bars & Smooth...","[delivery, pickup]"
4,Brentwood city,Brentwood,CA,37.9259,-121.697,Enishi Kitchen,"[Sushi Bars, Chinese, Japanese]",[delivery]


In [23]:
urban

Empty DataFrame
Columns: [Searched, City, State, Lat, Lng, Restaurant, Category, Transactions]
Index: []
